In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    import os
    os.chdir('/content')
    if not os.path.isdir('/content/EVA-2-Group/'):
        !git clone https://github.com/sambitdash/EVA-2-Group.git
    os.chdir('/content/EVA-2-Group/Session-19')
    !pwd
    
    !git config user.email "sambitdash@gmail.com"
    !git config user.name "Sambit Kumar Dash"
    !git config user.password "your password"
    !git status

In [2]:
import cv2
import os
import numpy as np
from os import listdir
from os.path import join

car_types = ['hatch', 'sedan', 'suv']

def resize_image(img, size=(64,64)):

    h, w = img.shape[:2]

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        c = img.shape[2]
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

spath, dpath = join('data', 'cars'), join('data', 'norm')

if not os.path.isdir(dpath):
    os.mkdir(dpath)

imgs = {}


for ct in car_types:
    sp, dp = join(spath, ct), join(dpath, ct)
    alen = 1024
    imgs[ct] = np.zeros((1024, 64, 64, 3))
    if not os.path.isdir(dp):
        os.mkdir(dp)
    tlen = 0
    for f in listdir(sp):
        sf, df = join(sp, f), join(dp, f)
        img = cv2.imread(sf)
        img = resize_image(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        tlen += 1
        if tlen > alen:
            imgs[ct] = np.append(imgs[ct], np.zeros((1024, 64, 64, 3)))
            alen += 1024
        imgs[ct][tlen-1] = img
    imgs[ct] = imgs[ct][:tlen]
    print(imgs[ct].shape)

(332, 64, 64, 3)
(410, 64, 64, 3)
(337, 64, 64, 3)


In [3]:
#%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

from tensorflow.keras import utils 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainx, testx = imgs['hatch'][:300,:,:,:], imgs['hatch'][300:,:,:,:]

trainy, testy = np.zeros(trainx.shape[0], dtype=float), np.zeros(testx.shape[0], dtype=float)

trainx = np.append(trainx, imgs['sedan'][:375,:,:,:], axis=0)
testx = np.append(testx, imgs['sedan'][375:,:,:,:], axis=0)

ltrain, ltest = trainx.shape[0] - trainy.shape[0], testx.shape[0] - testy.shape[0]

trainy, testy = np.append(trainy, np.ones(ltrain, dtype=float)), np.append(testy, np.ones(ltest, dtype=float))


print(trainx.shape, "y", trainy.shape)
print(testx.shape, 'y', testy.shape)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(trainx)

trainX, trainY = trainx, trainy
testX, testY   = testx, testy

for i in range(0):
    iterate = datagen.flow(trainx, trainy, batch_size=len(trainx), shuffle=True)
    x, y = iterate.next()
    trainX, trainY = np.append(trainX, x, axis=0), np.append(trainY, y, axis=0)

for i in range(0):
    iterate = datagen.flow(testx, testy, batch_size=len(testx), shuffle=True)
    x, y = iterate.next()
    testX, testY = np.append(testX, x, axis=0), np.append(testY, y, axis=0)


print(trainX.shape, "y", trainY.shape)
print(testX.shape, "y", testY.shape)

trainx, trainy = trainX, trainY
testx, testy   = testX, testY

trainx = trainx.astype('float32') / 255
testx  = testx.astype('float32') / 255

trainx_mean = np.mean(trainx, axis=(0, 1, 2))
trainx_std  = np.std(trainx, axis=(0, 1, 2))

print(trainx_mean, trainx_std)

trainx -= trainx_mean
trainx /= trainx_std

testx -= trainx_mean
testx /= trainx_std

trainX, trainY = trainx, trainy #utils.to_binary(trainy)
testX,  testY  = testx,  testy  #utils.to_binary(testy)

min_pix, max_pix = trainX.min(), trainX.max()

print(min_pix, max_pix)
print(testX.min(), testX.max())

print(trainX.shape, trainY.shape)

/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sambit/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

1.14.0
(675, 64, 64, 3) y (675,)
(67, 64, 64, 3) y (67,)
(675, 64, 64, 3) y (675,)
(67, 64, 64, 3) y (67,)
[0.35301414 0.34561545 0.34175763] [0.3740168  0.37273017 0.37292084]
-0.9438457 1.7650994
-0.9438457 1.7650994
(675, 64, 64, 3) (675,)


In [4]:
print(testY)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation, MaxPool2D
from tensorflow.keras.layers import add, Input, Dense, Flatten, GlobalAvgPool2D, GlobalAvgPool1D
from tensorflow.keras.initializers import zeros
from tensorflow.keras import regularizers

def ResConv(x, kernel=(3, 3), depth=32, maxpool=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if maxpool :
        x = MaxPool2D()(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def ResUnit(x, depth=32, maxpool=False):
    x = ResConv(x, depth=depth, maxpool=maxpool)
    x = ResConv(x, depth=depth)
    return x
    
def ResNetBlock(x, nunit, depth=32, maxpool=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    if maxpool:
        xskip = Conv2D(depth, (1, 1), strides=2, use_bias=False)(x)
    else: 
        xskip = x
    x = add([ResUnit(x, depth=depth, maxpool=maxpool), xskip])
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = add([ResUnit(x, depth=depth), x])
        x = add([ResUnit(x, depth=depth), x])
    return x

# Returns latent vector of 32 bytes
def ResNet9(x):
    x = Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False)(x)
    x = MaxPool2D((3, 3),  strides=2, padding='same')(x)
    
    nunits   = (2, 3, 2)
    maxpools = (False, True, True)
    depths   = (64, 32, 32)
    
    for i in range(3):
        x = ResNetBlock(x, nunits[i], depth=depths[i], maxpool=maxpools[i])
    x = GlobalAvgPool2D()(x)
    return x

#def D_init():
#    xin = Input(shape=(64, 64, 3))
#    x = ResNet9(xin)
#    x = Dense(1, use_bias=False, activation='sigmoid')(x)
#    return Model(xin, x)

def Q_init():
    xin = Input(shape=(64, 64, 3))
    x = ResNet9(xin)
    x = Dense(1, use_bias=False, activation='sigmoid')(x)
    return Model(xin, x)


def D_init():
    xin = Input(shape=(64, 64, 3))
    x = ResNet9(xin)
    x = Dense(1, use_bias=False, activation='sigmoid')(x)
    return Model(xin, x)


#    xin = Input(shape=(64, 64, 3))
#    x = Conv2D(8,(7, 7), use_bias=False, kernel_regularizer=regularizers.l2(0.01), padding='same')(xin)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = MaxPool2D(2,2)(x)
#    x = Conv2D(8,(7, 7), use_bias=False, kernel_regularizer=regularizers.l2(0.01), padding='same')(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = MaxPool2D(2,2)(x)
#    x = Conv2D(1,(7, 7), use_bias=False, kernel_regularizer=regularizers.l2(0.01), padding='same')(x)
#    x = BatchNormalization()(x)
#    x = Activation('relu')(x)
#    x = GlobalAvgPool2D()(x)
#    x = Flatten()(x)
#    x = Activation('sigmoid')(x)
#    return Model(xin, x)
    

In [6]:
from tensorflow.keras import Model
from tensorflow.keras.layers import UpSampling2D, Reshape, Conv2DTranspose

def InvResConv(x, kernel=(3, 3), depth=32, upscale=False):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if upscale :
        x = UpSampling2D((2,2))(x)
    x = Conv2D(depth, kernel, padding='same', use_bias=False)(x)
    return x

def InvResUnit(x, depth=32, upscale=False):
    x = InvResConv(x, depth=depth)
    x = InvResConv(x, depth=depth, upscale=upscale)
    return x
    
def InvResNetBlock(x, nunit, depth=32, upscale=False):
    assert nunit > 0, "Ensure there are at least 1 unit in the ResNet Block"
    nunit -= 1
    x = InvResUnit(x, depth=depth, upscale=upscale)
    if nunit >= 1:
        nunit -= 1
        for i in range(nunit):
            x = InvResUnit(x, depth=depth)
        x = InvResUnit(x, depth=depth)
    return x


def InvResNet9(x, size=(4,4)):
    depths   = (32, 32, 64)
    upscales = (True, True, False)
    nunits   = (2, 3, 2)

    x = UpSampling2D(size)(x)
    for i in range(3):
        x = InvResNetBlock(x, nunits[i], depth=depths[i], upscale=upscales[i])
    x = UpSampling2D((2,2))(x)
    x = Conv2DTranspose(3, (7, 7), strides=2, padding='same', use_bias=False)(x)
    return x

In [7]:
def G_init():
    xin = Input(shape=(32,), name="Input")
    x = Reshape((1,1,32))(xin)
    x = InvResNet9(x)
    x = Activation('relu')(x)
    model = Model(xin, x)
    return model

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = 8 #int(np.sqrt(s / r))
            h = 8 #int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = 0.0 #np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

datagen = ImageDataGenerator(
        fill_mode = 'constant',
        cval=0,
        rotation_range=20,
        width_shift_range=8,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=8,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        preprocessing_function=get_random_eraser(v_l=min_pix, v_h=max_pix, pixel_level=False)
)
datagen.fit(trainX)

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result)
    true_class = np.argmax(test_y)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['bacc'])+1),model_history.history['bacc'])
    axs[0].plot(range(1,len(model_history.history['val_bacc'])+1),model_history.history['val_bacc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['bacc'])+1),len(model_history.history['bacc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [10]:
BATCH_SIZE = 64
WT_DECAY   = 1e-5
MOMENTUM   = 0.90
LEARNING_RATE = 0.002

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras import backend as K

def l2_weights(model):
    l2 = 0
    for layer in model.layers: 
        if isinstance(layer, Model):
            continue
        wt = layer.weights
        if len(wt) > 0:
            l2 += K.sum(K.pow(wt, 2))
    return l2

def reg_loss(model):
    def rloss(y_true, y_pred):
        return WT_DECAY*l2_weights(model)
    return rloss

def loss_with_regularization(model):
    def loss(y_true, y_pred):
        return binary_crossentropy(y_true, y_pred, True) + reg_loss(model)(y_true, y_pred)
    return loss

def bacc(y_true, y_pred):
    y_pred = K.cast(y_pred > 0.5, dtype=y_true.dtype)
    return K.mean(K.equal(y_true, y_pred), axis=-1)

Q = Q_init()
optimizer = SGD(lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)
Q.compile(optimizer=optimizer, loss=loss_with_regularization(Q), metrics=[bacc, reg_loss(Q)])
Q.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 16, 16, 64)   0           conv2d[0][0]                     
________________________________________________________________________________

In [11]:
D = D_init()
D.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
D.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 32, 32, 64)   9408        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 16, 16, 64)   0           conv2d_17[0][0]                  
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 16, 16, 64)   256         max_pooling2d_3[0][0]            
____________________________________________________________________________________________

In [12]:
G = G_init()

G.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
reshape (Reshape)            (None, 1, 1, 32)          0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 4, 4, 32)          128       
_________________________________________________________________
activation_28 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 4, 4, 32)          9216      
_________________________________________________________________
batch_normalization_29 (Batc (None, 4, 4, 32)          128 

In [13]:
def gan_loss(model):
    def gloss(y_true, y_pred):
        return binary_crossentropy(y_true, y_pred) + 1e-3*l2_weights(model)
    return gloss
    
def define_gan(g_model, d_model, q_model):
    d_model.trainable = False
    q_model.trainable = False
    d_output = d_model(g_model.output)
    q_output = q_model(g_model.output)
    gan = Model(g_model.input, [d_output, q_output])
    #opt = Adam(lr=0.0002, beta_1=0.5)
    gan.compile(loss=[gan_loss(gan), 'binary_crossentropy'], optimizer='adam', metrics=['acc'])
    return gan

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.models import load_model

mcp = ModelCheckpoint("q.hf5", monitor='val_bacc', save_weights_only=True, save_best_only=True)

class MyEarlyStopping(Callback):
    def __init__(self, size):
        self.baseline = size*0.5
        super().__init__()
    def on_epoch_begin(self, epoch, logs=None):
        self.correct  = 0
    def on_batch_end(self, batch, logs=None):
        self.correct += logs['size']*logs['acc']
        if self.correct > self.baseline:
            self.model.stop_training = True

for i in range(100):
    print("Iteration Count ##### ", i)
    Q.fit_generator(datagen.flow(trainX, trainY, batch_size=BATCH_SIZE), epochs=100, shuffle=True, 
                    validation_data=(testX, testY), callbacks=[mcp], verbose=0)
    
    qres = Q.evaluate(testX, testY)
    print(qres)
    
    Q.save_weights("qb.hf5")
    
    Q.load_weights("q.hf5")
    

    func = K.function(Q.input, Q.layers[-2].output)
    Gtrain, Gval = func(trainX), func(testX)
    
    fake = np.random.randn(*trainX.shape) if i == 0 else K.function(gan.input, G.output)(Gtrain)

    Dx = np.append(trainX, fake, axis=0)
    Dy = np.append(np.ones((trainX.shape[0],)), np.zeros((fake.shape[0],)))
    

    dres = D.evaluate(Dx, Dy)
    print(dres)

    count = 0 
    while dres[1] < 0.60 and count < 20:
        s = np.arange(Dx.shape[0])
        np.random.shuffle(s)
        Dx, Dy = Dx[s], Dy[s]
        D.fit(Dx, Dy, epochs=1, batch_size=32, 
              validation_data=(testX, np.ones((testX.shape[0],))), callbacks=[MyEarlyStopping(len(Dx))], verbose=0)
        dres = D.evaluate(Dx, Dy)
        count += 1
        print(dres)
    
    gan = define_gan(G, D, Q)
    Q.trainable = True
    
    gan.fit(Gtrain, [np.zeros((Gtrain.shape[0],)), trainY], batch_size=BATCH_SIZE, epochs=100, verbose=0)
    
    gres = gan.evaluate(Gval, [np.zeros((Gval.shape[0],)), testY])
    
    print(gres)
    
    Q.load_weights("qb.hf5")

Iteration Count #####  0
67/67 [==============================] - 0s 303us/sample - loss: 0.6693 - bacc: 0.6866 - rloss: 0.0253
[0.6692899977093312, 0.6865672, 0.025288379]
1350/1350 [==============================] - 0s 264us/sample - loss: 0.9889 - acc: 0.4956
[0.9888695262979578, 0.49555555]
1350/1350 [==============================] - 0s 146us/sample - loss: 12.4398 - acc: 0.5037
[12.43978571150038, 0.5037037]
1350/1350 [==============================] - 0s 144us/sample - loss: 3.4072 - acc: 0.6044
[3.407194035847982, 0.60444444]
67/67 [==============================] - 1s 9ms/sample - loss: 2.3615 - model_1_loss: 1.1148 - model_loss: 0.8830 - model_1_acc: 1.0000 - model_acc: 0.6567
[2.361523414725688, 1.114797, 0.8829722, 1.0, 0.6567164]
Iteration Count #####  1
67/67 [==============================] - 0s 309us/sample - loss: 0.7091 - bacc: 0.6418 - rloss: 0.0294
[0.70913951432527, 0.64179105, 0.029427836]
1350/1350 [==============================] - 0s 111us/sample - loss: 3.4072

1350/1350 [==============================] - 0s 165us/sample - loss: 0.0292 - acc: 1.0000
[0.02924386503243888, 1.0]
67/67 [==============================] - 1s 19ms/sample - loss: 5.3097 - model_1_loss: 3.6135 - model_loss: 1.1838 - model_1_acc: 1.0000 - model_acc: 0.7463
[5.309658029186192, 3.613477, 1.1838018, 1.0, 0.74626863]
Iteration Count #####  15
67/67 [==============================] - 0s 331us/sample - loss: 0.7649 - bacc: 0.6866 - rloss: 0.1002
[0.7649452882026558, 0.6865672, 0.10020178]
1350/1350 [==============================] - 0s 141us/sample - loss: 5.6778e-04 - acc: 1.0000
[0.0005677777325682756, 1.0]
67/67 [==============================] - 1s 20ms/sample - loss: 5.2665 - model_1_loss: 3.5852 - model_loss: 1.1736 - model_1_acc: 1.0000 - model_acc: 0.7761
[5.266549245635075, 3.5852134, 1.1735594, 1.0, 0.7761194]
Iteration Count #####  16
67/67 [==============================] - 0s 352us/sample - loss: 0.7650 - bacc: 0.6716 - rloss: 0.1066
[0.7649789781712774, 0.67164

67/67 [==============================] - 0s 368us/sample - loss: 0.7951 - bacc: 0.7015 - rloss: 0.1410
[0.7951427840474826, 0.70149255, 0.14097361]
1350/1350 [==============================] - 0s 188us/sample - loss: 0.0090 - acc: 1.0000
[0.008994016993307957, 1.0]
67/67 [==============================] - 2s 32ms/sample - loss: 9.6756 - model_1_loss: 3.6028 - model_loss: 7.1649 - model_1_acc: 1.0000 - model_acc: 0.5522
[9.675619993636857, 3.602789, 7.164889, 1.0, 0.5522388]
Iteration Count #####  31
67/67 [==============================] - 0s 355us/sample - loss: 0.7876 - bacc: 0.7015 - rloss: 0.1404
[0.7876449240677392, 0.70149255, 0.14041826]
1350/1350 [==============================] - 0s 193us/sample - loss: 0.0064 - acc: 1.0000
[0.006355766929617083, 1.0]
67/67 [==============================] - 2s 35ms/sample - loss: 5.1837 - model_1_loss: 3.6334 - model_loss: 1.0811 - model_1_acc: 1.0000 - model_acc: 0.7015
[5.1836625390978, 3.6333704, 1.0810503, 1.0, 0.70149255]
Iteration Count

In [ ]:
func = K.function(Q.input, Q.layers[-2].output)
Gtrain, Gval = func(trainX), func(testX)

In [ ]:
func1 = K.function(G.input, G.output)
testfake = func1(Gval)
len(testfake)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dres